In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os

In [3]:
# os.chdir("/content/drive/MyDrive/Final year/Personal Project/youtube_thing/Text-Summarizer")
os.chdir("../")

In [4]:
%pwd

'/content/drive/MyDrive/Final year/Personal Project/youtube_thing/Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [6]:
# !pip install ensure
# !pip install sentencepiece
# # Run these commands to fix the accelerate module dependencies
# !pip install --upgrade accelerate
# !pip uninstall -y transformers accelerate
# !pip install transformers accelerate

Found existing installation: transformers 4.36.1
Uninstalling transformers-4.36.1:
  Successfully uninstalled transformers-4.36.1
Found existing installation: accelerate 0.25.0
Uninstalling accelerate-0.25.0:
  Successfully uninstalled accelerate-0.25.0
  Using cached transformers-4.36.1-py3-none-any.whl (8.3 MB)
  Using cached accelerate-0.25.0-py3-none-any.whl (265 kB)


In [7]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])



    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [9]:
# !pip install datasets

In [10]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

In [11]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config



    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        #loading data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # trainer_args = TrainingArguments(
        #     output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
        #     per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
        #     weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
        #     evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
        #     gradient_accumulation_steps=self.config.gradient_accumulation_steps
        # )


        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
        )

        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["test"],
                  eval_dataset=dataset_samsum_pt["validation"])

        trainer.train()

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [12]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


In [14]:
!git config --global user.name "dhyutin"
!git config --global user.email "ced19i027@iiitdm.ac.in"

In [15]:
!git remote add origin https://github.com/dhyutin/Text-Summarizer.git

error: remote origin already exists.


In [21]:
# !git lfs install
!git lfs ls-files


0d4f75ac02 * artifacts/model_trainer/pegasus-samsum-model/model.safetensors


In [24]:
# !git rm --cached path/to/largefile


fatal: pathspec 'path/to/largefile' did not match any files


In [26]:
# !chmod +x .git/hooks/post-commit
# !chmod +x .git/hooks/pre-push


In [34]:
# !git remote add origin https://github.com/dhyutin/Text-Summarizer.git
# !git branch -M main
# !git add -u
# !git commit -m "to collab - Model Training"
# !git push origin main

error: remote origin already exists.
[main c6ab2cb] to collab - Model Training
 609 files changed, 1 insertion(+), 315391 deletions(-)
 delete mode 100644 artifacts/data_ingestion/data.zip
 delete mode 100644 artifacts/data_ingestion/samsum-test.csv
 delete mode 100644 artifacts/data_ingestion/samsum-train.csv
 delete mode 100644 artifacts/data_ingestion/samsum-validation.csv
 delete mode 100644 artifacts/data_ingestion/samsum_dataset/dataset_dict.json
 delete mode 100644 artifacts/data_ingestion/samsum_dataset/test/cache-79bf5921395b6c28.arrow
 delete mode 100644 artifacts/data_ingestion/samsum_dataset/test/cache-e536525996ed4f3b.arrow
 delete mode 100644 artifacts/data_ingestion/samsum_dataset/test/data-00000-of-00001.arrow
 delete mode 100644 artifacts/data_ingestion/samsum_dataset/test/dataset_info.json
 delete mode 100644 artifacts/data_ingestion/samsum_dataset/test/state.json
 delete mode 100644 artifacts/data_ingestion/samsum_dataset/train/cache-02eed84ffcfcaa58.arrow
 delete mo